In [1]:
from typing import Sequence, Union, Tuple, Any, Dict, Optional

import clingo

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            symbols = model.symbols(atoms=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models


In [3]:
instance = """

circle(c1). representation(c1, @geometry_id(c1)).
circle(c2). representation(c2, @geometry_id(c2)).

"""

In [4]:
reasoning = """

region(C) :- circle(C).

extent(A, @extent(R)) :-
  region(A), representation(A, R).

size(Cmp, A, B) :-
  region(A), region(B),
  @compare_extents(A, B) = Cmp.

topology(part_of, A, A) :- region(A).
size(eq, A, A) :- region(A).
size(lt, A, B) :- size(gt, B, A).
-topology(part_of, A, B) :- region(A), region(B), not size(eq, A, B), not size(lt, A, B).

"""

In [5]:
class SpatialReasoningContext:

    def __init__(self, geometry_db: Optional[Dict[Union[str, int], Dict[str, Any]]]):
        self.geometry_db = geometry_db
        if self.geometry_db is None:
            self.geometry_db: dict = {}

    def __repr__(self):
        return repr(self.geometry_db)

    def __str__(self):
        return str(self.geometry_db)

    def __get_db_entry(self, representation: clingo.Symbol) -> Tuple[Union[str, int], dict]:
        py_representation = SpatialReasoningContext.get_py_representation(representation)
        if py_representation not in self.geometry_db:
            raise KeyError(
                "{} is not in the geometry_db".format(py_representation))
        return py_representation, self.geometry_db[py_representation]

    def __extent(self, py_representation: Union[str, int]):
        pass

    def geometry_id(self, geometry_object: clingo.Symbol) -> clingo.Symbol:
        py_representation = SpatialReasoningContext.get_py_representation(geometry_object)
        if py_representation not in self.geometry_db:
            self.geometry_db[py_representation] = {}
        assert py_representation in self.geometry_db, "{} should be in geometry_db".format(py_representation)
        return geometry_object

    def extent(self, representation: clingo.Symbol):
        py_representation, entry = self.__get_db_entry(representation)
        if not 'extent' in entry:
            self.__extent(py_representation)
        return clingo.String(entry['extent'])

    def compare_extents(self, representation1: clingo.Symbol, representation2: clingo.Symbol):
        py_representation1, entry1 = self.__get_db_entry(representation1)
        py_representation2, entry2 = self.__get_db_entry(representation2)
        assert 'extent' in entry1, "{} should have an extent".format(py_representation1)
        assert 'extent' in entry2, "{} should have an extent".format(py_representation2)

        extent1 = entry1['extent']
        extent2 = entry2['extent']

        if extent1 == extent2:
            return clingo.Function('eq')
        elif extent1 < extent2:
            return clingo.Function('lt')
        else:
            assert extent1 > extent2, "Unexpected case: compare_extents({},{})".format(extent1, extent2)
            return clingo.Function('gt')



    @staticmethod
    def get_py_representation(sym: clingo.Symbol):
        if sym.type == clingo.SymbolType.String:
            return sym.string
        elif sym.type == clingo.SymbolType.Number:
            return sym.number

        else:
            return str(sym)



In [6]:
initial_db = dict(
    c1=dict(type='circle', x_coordinate=0.0, y_coordinate=0.0, extent='pi*4'),
    c2=dict(type='circle', x_coordinate=0.0, y_coordinate=0.0, extent='pi*1')
)
spc = SpatialReasoningContext(initial_db)
spc

{'c1': {'type': 'circle', 'x_coordinate': 0.0, 'y_coordinate': 0.0, 'extent': 'pi*4'}, 'c2': {'type': 'circle', 'x_coordinate': 0.0, 'y_coordinate': 0.0, 'extent': 'pi*1'}}

In [7]:
solve([
    reasoning,
    instance
], sep='\n', grounding_context=spc);

Answer 1: {
circle(c1)
circle(c2)
region(c1)
region(c2)
extent(c1,"pi*4")
extent(c2,"pi*1")
representation(c1,c1)
representation(c2,c2)
size(eq,c1,c1)
size(eq,c2,c2)
size(gt,c1,c2)
size(lt,c2,c1)
topology(part_of,c1,c1)
topology(part_of,c2,c2)
-topology(part_of,c1,c2)
}
SAT 1
